In [1]:
from exact_synthesis.prec import synthesize_z_rotation
from exact_synthesis.prec import evaluate_gate_sequence, rz_mp
from exact_synthesis.prec import d
from exact_synthesis.synthesis import transpile_ft_sigma
import mpmath
from mpmath import mp

/home/jkr/projects/topological-quantum-compiling/src/exact_synthesis/numberTheory.py:366: SyntaxWarning: invalid escape sequence '\i'
  """


In [2]:
input = lambda k : 2 * mpmath.pi * k / (10**3)
error = lambda k : 10**(-k)

C,U = synthesize_z_rotation(input(1), error(4))
print(len(C))
print(C)

Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
FOUND SOLUTION
1
56
[WIGate(power=1), TGate(power=6), FGate(), TGate(power=5), FGate(), TGate(power=9), FGate(), TGate(power=9), FGate(), TGate(power=9), FGate(), TGate(power=9), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=4), FGate(), TGate(power=1), FGate(), TGate(power=1), FGate(), TGate(power=2), FGate(), TGate(power=4), FGate(), TGate(power=1), FGate(), TGate(power=8), FGate(), TGate(power=9), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=2), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=4), FGate(), TGate(power=8), FGate(), TGate(power=2), FGate(), TGate(power=4), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=3)]


In [3]:
dist = d(input(1),U.u, U.v)

dist < error(4)
dist - error(4)

mpf('0.0098919997541270805')

In [4]:
mpmath.dps = 200
C,U = synthesize_z_rotation(input(10), 1e-5)
print(len(C))
dist=d(input(4),U.u, U.v)
dist

Equations is not going to be solvable
FOUND SOLUTION
1
58


mpf('0.0094060448186145569')

In [5]:
print(C)

[WIGate(power=9), TGate(power=6), FGate(), TGate(power=8), FGate(), TGate(power=8), FGate(), TGate(power=1), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=1), FGate(), TGate(power=8), FGate(), TGate(power=9), FGate(), TGate(power=2), FGate(), TGate(power=1), FGate(), TGate(power=8), FGate(), TGate(power=2), FGate(), TGate(power=9), FGate(), TGate(power=5), FGate(), TGate(power=8), FGate(), TGate(power=5), FGate(), TGate(power=1), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=1), FGate(), TGate(power=9), FGate(), TGate(power=1), FGate(), TGate(power=2), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=8), FGate(), TGate(power=6), FGate(), TGate(power=7)]


In [7]:
CC =transpile_ft_sigma(C)
len(CC)

660

In [3]:
from mpmath import matrix

A = matrix([[0,0],[0,0]])

print(type(A))

<class 'mpmath.matrices.matrices.matrix'>


In [ ]:
import numpy as np
from mpmath import mpc, mp, atan2, sqrt, cos, sin, pi

mp.dps = 30  # arbitrary precision

def mag(z):
    return abs(z)

def phase(z):
    return atan2(z.imag, z.real)

def cis(x):
    return mpc(cos(x), sin(x))

def adj(z):
    return mpc(z.real, -z.imag)

def euler_angles(U):
    """
    Decompose U ∈ U(2) as e^{iα} · Rz(β) · Rx(γ) · Rz(δ)
    """
    (a, b), (c, d) = U.tolist()
    a = mpc(a)
    b = mpc(b)
    c = mpc(c)
    d = mpc(d)

    i = mpc(0, 1)
    alpha = phase(a*d - b*c) / 2
    gamma = 2 * atan2(mag(b), mag(a))
    delta = phase(b * d * i * adj(a*d - b*c))
    beta = 2 * phase(d * cis(-alpha - delta / 2) + c * cis(-alpha + delta / 2) * i)

    return float(alpha), float(beta), float(gamma), float(delta)


def global_phase(U):
    """Compute the global phase angle from the determinant"""
    det = np.linalg.det(U)
    return np.angle(det) / 2


def _decompose_U(U, tol=1e-12):
    """
    Given a 2x2 unitary matrix U, return Euler angles for the decomposition:
        U = e^{iα} · Rz(β) · Rx(γ) · Rz(δ)

    Returns a dictionary with keys:
      - delta (global phase α)
      - alpha, beta, gamma, delta (Euler angles)
      - form: description of decomposition
    """
    assert U.shape == (2, 2), "Matrix must be 2x2"
    assert np.allclose(U.conj().T @ U, np.eye(2), atol=1e-12), "Matrix must be unitary"

    delta = global_phase(U)
    U_prime = U * np.exp(-1j * delta)

    if np.abs(U_prime[0, 0]) > tol:
        alpha, beta, gamma, delta_ = euler_angles(U_prime)
        return {
            "global_phase": delta,
            "alpha": alpha,
            "beta": beta,
            "gamma": gamma,
            "delta": delta_,
            "form": "Rz-Rx-Rz",
        }
    else:
        X = np.array([[0, 1], [1, 0]])
        Rz_phi = U_prime @ X
        diag = np.diag(Rz_phi)
        phi = np.angle(0.5 * diag[0])
        return {
            "global_phase": delta,
            "phi": phi,
            "form": "Rz-X",
        }

from scipy.linalg import expm

theta = np.pi / 4
U = expm(1j * theta * np.array([[0, 1], [1, 0]]))
result = _decompose_U(U)
print(result)


{'global_phase': np.float64(0.0), 'alpha': 0.0, 'beta': -3.141592653589793, 'gamma': 1.5707963267948966, 'delta': 3.141592653589793, 'form': 'Rz-Rx-Rz'}
